In [ ]:
import tensorflow as tf
from scipy.stats import gaussian_kde
import numpy as np

tf.enable_eager_execution()

g = tf.Graph()
with g.as_default():
    N = 10000
    Wafer1 = np.random.normal(loc = 0.0, scale = 1.0, size = N)
    Wafer2 = np.random.normal(loc = 2.0, scale = 1.0, size = N)
    
    pdf1 = gaussian_kde(Wafer1)
    pdf2 = gaussian_kde(Wafer2)
    
    pdf1.set_bandwidth(bw_method=pdf1.factor / 3.)
    pdf2.set_bandwidth(bw_method=pdf2.factor / 3.)

    def lin_model(x, w, b):
        return tf.add(tf.scalar_mul(w, x), b)
    
    def error_function(prediction, teacher):
#         print(np.shape(prediction), np.shape(teacher))
        return tf.reduce_mean(tf.square(prediction-teacher))

    def pdf_wrapper(y):
        temp = y.numpy()
        result = pdf2(temp)
        result = tf.convert_to_tensor(result, dtype=tf.float32)
        return result
    
    def generate_grid_data():
        w_min, w_max = -3, 3
        b_min, b_max = -3, 3
        
        ww, bb = np.mgrid[w_min:w_max:100j, b_min:b_max:100j]
        pos_grid = np.vstack([ww.ravel(), bb.ravel()]) # 2 x (100 x 100)
        return pos_grid

    # define variables or constants
    X_test = np.linspace(min(Wafer1), max(Wafer1), num=N)
    x = tf.constant(X_test, dtype=tf.float32)
    w = tf.Variable(1.0, name='w')
    b = tf.Variable(1.9, name='b')
    
    # define pdf2 wrapper
    y_hypo = lin_model(x, w, b)
    ypdf_hypo = tf.py_function(pdf_wrapper, [y_hypo], tf.float32)
    
    
    # calculate real values from pdf1 and store them
    y_real = tf.constant(pdf1(X_test), dtype=tf.float32)
    
    # generate w,b grid
    grid_wb = generate_grid_data()
    
    # min error, and corresponding w and b
    min_error = None
    min_w = None
    min_b = None
    
    # start tensorflow
    init = tf.global_variables_initializer()
    s = tf.Session(graph=g)
    s.run(init)
    for i in range(N):
        current_w = grid_wb[0][i]
        current_b = grid_wb[1][i]
        
        # assign w and b
        opw = w.assign(current_w)
        s.run(opw)
        opb = w.assign(current_b)
        s.run(opb)
        
        print(s.run(w), s.run(b))
        
        y_predict = s.run(ypdf_hypo)
        cost = error_function(y_predict, y_real)
        err = s.run(cost)
        print("error:", err)
        print("direct value:", pdf2(X_test[0] + 1.9), "predict value:", y_predict[0])
        if min_error is None or min_error > err: # Record min err and its position
            min_error = err
            min_w = current_w
            min_b = current_b
    
    s.close()
    print("final:", min_w, min_b, min_error)
    Y1 = pdf1(X_test)
    X_hat = min_w * X_test + min_b
    Y2 = pdf2(X_hat)
    import matplotlib.pyplot as plt
    plt.hist(Y1,bins=30)
    plt.hist(Y2, bins=30)
    plt.show()

Instructions for updating:
Colocations handled automatically by placer.
-3.0 1.9
error: 0.016071545
direct value: [6.144126e-07] predict value: 0.0
-2.939394 1.9
error: 0.015716165
direct value: [6.144126e-07] predict value: 0.0
-2.878788 1.9
error: 0.0153499935
direct value: [6.144126e-07] predict value: 0.0
-2.8181818 1.9
error: 0.01497266
direct value: [6.144126e-07] predict value: 0.0
-2.7575758 1.9
error: 0.014583791
direct value: [6.144126e-07] predict value: 0.0
-2.6969697 1.9
error: 0.014183014
direct value: [6.144126e-07] predict value: 0.0
-2.6363637 1.9
error: 0.013769912
direct value: [6.144126e-07] predict value: 0.0
-2.5757575 1.9
error: 0.013344034
direct value: [6.144126e-07] predict value: 0.0
-2.5151515 1.9
error: 0.01290488
direct value: [6.144126e-07] predict value: 0.0
-2.4545455 1.9
error: 0.012451907
direct value: [6.144126e-07] predict value: 0.0
-2.3939395 1.9
error: 0.011984572
direct value: [6.144126e-07] predict value: 0.0
-2.3333333 1.9
error: 0.0115024345


error: 0.01384324
direct value: [6.144126e-07] predict value: 0.0
2.6969697 1.9
error: 0.01425534
direct value: [6.144126e-07] predict value: 0.0
2.7575758 1.9
error: 0.014654781
direct value: [6.144126e-07] predict value: 0.0
2.8181818 1.9
error: 0.015042252
direct value: [6.144126e-07] predict value: 0.0
2.878788 1.9
error: 0.015418289
direct value: [6.144126e-07] predict value: 0.0
2.939394 1.9
error: 0.015783267
direct value: [6.144126e-07] predict value: 0.0
3.0 1.9
error: 0.016137475
direct value: [6.144126e-07] predict value: 0.0
-3.0 1.9
error: 0.016071545
direct value: [6.144126e-07] predict value: 0.0
-2.939394 1.9
error: 0.015716165
direct value: [6.144126e-07] predict value: 0.0
-2.878788 1.9
error: 0.0153499935
direct value: [6.144126e-07] predict value: 0.0
-2.8181818 1.9
error: 0.01497266
direct value: [6.144126e-07] predict value: 0.0
-2.7575758 1.9
error: 0.014583791
direct value: [6.144126e-07] predict value: 0.0
-2.6969697 1.9
error: 0.014183014
direct value: [6.1441

error: 0.012524426
direct value: [6.144126e-07] predict value: 0.0
2.5151515 1.9
error: 0.012978189
direct value: [6.144126e-07] predict value: 0.0
2.5757575 1.9
error: 0.013417715
direct value: [6.144126e-07] predict value: 0.0
2.6363637 1.9
error: 0.01384324
direct value: [6.144126e-07] predict value: 0.0
2.6969697 1.9
error: 0.01425534
direct value: [6.144126e-07] predict value: 0.0
2.7575758 1.9
error: 0.014654781
direct value: [6.144126e-07] predict value: 0.0
2.8181818 1.9
error: 0.015042252
direct value: [6.144126e-07] predict value: 0.0
2.878788 1.9
error: 0.015418289
direct value: [6.144126e-07] predict value: 0.0
2.939394 1.9
error: 0.015783267
direct value: [6.144126e-07] predict value: 0.0
3.0 1.9
error: 0.016137475
direct value: [6.144126e-07] predict value: 0.0
-3.0 1.9
error: 0.016071545
direct value: [6.144126e-07] predict value: 0.0
-2.939394 1.9
error: 0.015716165
direct value: [6.144126e-07] predict value: 0.0
-2.878788 1.9
error: 0.0153499935
direct value: [6.144126

2.2727273 1.9
error: 0.011078906
direct value: [6.144126e-07] predict value: 0.0
2.3333333 1.9
error: 0.011574562
direct value: [6.144126e-07] predict value: 0.0
2.3939395 1.9
error: 0.012056485
direct value: [6.144126e-07] predict value: 0.0
2.4545455 1.9
error: 0.012524426
direct value: [6.144126e-07] predict value: 0.0
2.5151515 1.9
error: 0.012978189
direct value: [6.144126e-07] predict value: 0.0
2.5757575 1.9
error: 0.013417715
direct value: [6.144126e-07] predict value: 0.0
2.6363637 1.9
error: 0.01384324
direct value: [6.144126e-07] predict value: 0.0
2.6969697 1.9
error: 0.01425534
direct value: [6.144126e-07] predict value: 0.0
2.7575758 1.9
error: 0.014654781
direct value: [6.144126e-07] predict value: 0.0
2.8181818 1.9
error: 0.015042252
direct value: [6.144126e-07] predict value: 0.0
2.878788 1.9
error: 0.015418289
direct value: [6.144126e-07] predict value: 0.0
2.939394 1.9
error: 0.015783267
direct value: [6.144126e-07] predict value: 0.0
3.0 1.9
error: 0.016137475
direc

2.090909 1.9
error: 0.009510672
direct value: [6.144126e-07] predict value: 0.0
2.1515152 1.9
error: 0.010046974
direct value: [6.144126e-07] predict value: 0.0
2.2121212 1.9
error: 0.0105697075
direct value: [6.144126e-07] predict value: 0.0
2.2727273 1.9
error: 0.011078906
direct value: [6.144126e-07] predict value: 0.0
2.3333333 1.9
error: 0.011574562
direct value: [6.144126e-07] predict value: 0.0
2.3939395 1.9
error: 0.012056485
direct value: [6.144126e-07] predict value: 0.0
2.4545455 1.9
error: 0.012524426
direct value: [6.144126e-07] predict value: 0.0
2.5151515 1.9
error: 0.012978189
direct value: [6.144126e-07] predict value: 0.0
2.5757575 1.9
error: 0.013417715
direct value: [6.144126e-07] predict value: 0.0
2.6363637 1.9
error: 0.01384324
direct value: [6.144126e-07] predict value: 0.0
2.6969697 1.9
error: 0.01425534
direct value: [6.144126e-07] predict value: 0.0
2.7575758 1.9
error: 0.014654781
direct value: [6.144126e-07] predict value: 0.0
2.8181818 1.9
error: 0.0150422

...

1.060606 1.9
error: 0.0004385345
direct value: [6.144126e-07] predict value: 3.002023e-18
1.1212121 1.9
error: 0.00063000235
direct value: [6.144126e-07] predict value: 7.207352e-38
1.1818181 1.9
error: 0.0009535693
direct value: [6.144126e-07] predict value: 0.0
1.2424242 1.9
error: 0.001373012
direct value: [6.144126e-07] predict value: 0.0
1.3030303 1.9
error: 0.0018624096
direct value: [6.144126e-07] predict value: 0.0
1.3636364 1.9
error: 0.0023977614
direct value: [6.144126e-07] predict value: 0.0
1.4242424 1.9
error: 0.0029669516
direct value: [6.144126e-07] predict value: 0.0
1.4848485 1.9
error: 0.0035612327
direct value: [6.144126e-07] predict value: 0.0
1.5454545 1.9
error: 0.004169651
direct value: [6.144126e-07] predict value: 0.0
1.6060606 1.9
error: 0.004781392
direct value: [6.144126e-07] predict value: 0.0
1.6666666 1.9
error: 0.0053939316
direct value: [6.144126e-07] predict value: 0.0
1.7272727 1.9
error: 0.006006866
direct value: [6.144126e-07] predict value: 0.0
1.

0.8787879 1.9
error: 0.0010607541
direct value: [6.144126e-07] predict value: 0.0015772583
0.93939394 1.9
error: 0.0006059452
direct value: [6.144126e-07] predict value: 0.0010300134
1.0 1.9
error: 0.0004261345
direct value: [6.144126e-07] predict value: 6.144163e-07
1.060606 1.9
error: 0.0004385345
direct value: [6.144126e-07] predict value: 3.002023e-18
1.1212121 1.9
error: 0.00063000235
direct value: [6.144126e-07] predict value: 7.207352e-38
1.1818181 1.9
error: 0.0009535693
direct value: [6.144126e-07] predict value: 0.0
1.2424242 1.9
error: 0.001373012
direct value: [6.144126e-07] predict value: 0.0
1.3030303 1.9
error: 0.0018624096
direct value: [6.144126e-07] predict value: 0.0
1.3636364 1.9
error: 0.0023977614
direct value: [6.144126e-07] predict value: 0.0
1.4242424 1.9
error: 0.0029669516
direct value: [6.144126e-07] predict value: 0.0
1.4848485 1.9
error: 0.0035612327
direct value: [6.144126e-07] predict value: 0.0
1.5454545 1.9
error: 0.004169651
direct value: [6.144126e-0

0.6969697 1.9
error: 0.0051224334
direct value: [6.144126e-07] predict value: 0.007797509
0.75757575 1.9
error: 0.003209928
direct value: [6.144126e-07] predict value: 0.0043832674
0.8181818 1.9
error: 0.0019100584
direct value: [6.144126e-07] predict value: 0.0033649146
0.8787879 1.9
error: 0.0010607541
direct value: [6.144126e-07] predict value: 0.0015772583
0.93939394 1.9
error: 0.0006059452
direct value: [6.144126e-07] predict value: 0.0010300134
1.0 1.9
error: 0.0004261345
direct value: [6.144126e-07] predict value: 6.144163e-07
1.060606 1.9
error: 0.0004385345
direct value: [6.144126e-07] predict value: 3.002023e-18
1.1212121 1.9
error: 0.00063000235
direct value: [6.144126e-07] predict value: 7.207352e-38
1.1818181 1.9
error: 0.0009535693
direct value: [6.144126e-07] predict value: 0.0
1.2424242 1.9
error: 0.001373012
direct value: [6.144126e-07] predict value: 0.0
1.3030303 1.9
error: 0.0018624096
direct value: [6.144126e-07] predict value: 0.0
1.3636364 1.9
error: 0.0023977614

0.5151515 1.9
error: 0.01661003
direct value: [6.144126e-07] predict value: 0.040938515
0.57575756 1.9
error: 0.0115600955
direct value: [6.144126e-07] predict value: 0.0292137
0.6363636 1.9
error: 0.007837652
direct value: [6.144126e-07] predict value: 0.014452506
0.6969697 1.9
error: 0.0051224334
direct value: [6.144126e-07] predict value: 0.007797509
0.75757575 1.9
error: 0.003209928
direct value: [6.144126e-07] predict value: 0.0043832674
0.8181818 1.9
error: 0.0019100584
direct value: [6.144126e-07] predict value: 0.0033649146
0.8787879 1.9
error: 0.0010607541
direct value: [6.144126e-07] predict value: 0.0015772583
0.93939394 1.9
error: 0.0006059452
direct value: [6.144126e-07] predict value: 0.0010300134
1.0 1.9
error: 0.0004261345
direct value: [6.144126e-07] predict value: 6.144163e-07
1.060606 1.9
error: 0.0004385345
direct value: [6.144126e-07] predict value: 3.002023e-18
1.1212121 1.9
error: 0.00063000235
direct value: [6.144126e-07] predict value: 7.207352e-38
1.1818181 1.

0.33333334 1.9
error: 0.04128297
direct value: [6.144126e-07] predict value: 0.15841383
0.3939394 1.9
error: 0.031502437
direct value: [6.144126e-07] predict value: 0.11044112
0.45454547 1.9
error: 0.023233468
direct value: [6.144126e-07] predict value: 0.071930006
0.5151515 1.9
error: 0.01661003
direct value: [6.144126e-07] predict value: 0.040938515
0.57575756 1.9
error: 0.0115600955
direct value: [6.144126e-07] predict value: 0.0292137
0.6363636 1.9
error: 0.007837652
direct value: [6.144126e-07] predict value: 0.014452506
0.6969697 1.9
error: 0.0051224334
direct value: [6.144126e-07] predict value: 0.007797509
0.75757575 1.9
error: 0.003209928
direct value: [6.144126e-07] predict value: 0.0043832674
0.8181818 1.9
error: 0.0019100584
direct value: [6.144126e-07] predict value: 0.0033649146
0.8787879 1.9
error: 0.0010607541
direct value: [6.144126e-07] predict value: 0.0015772583
0.93939394 1.9
